In [284]:
import pandas as pd

# load the necessary Pulp functions
from pulp import LpProblem, LpMinimize, LpVariable, lpSum

In [285]:

# read data from excel file 'project_plan.xlsx'
# file is stored at '002. Input Tables'

input_file_path = '002. Input Tables/project_plan.xlsx'

# read data from excel file 'project_plan_simple.xlsx'
# if NaN then replace by None,
# if column is string then remove trailing characters
def read_data(file_path):
    df = pd.read_excel(file_path, keep_default_na=False)
    return df

# read data from excel file 'project_plan_simple.xlsx'
df = read_data(input_file_path)

# display data as markdown table
print(df.to_markdown())

# export markdown table to '003. Output Files/table_01_project_plan.md'
output_file_path = '003. Output Files/table_01_project_plan.md'
with open(output_file_path, 'w') as f:
    f.write(df.to_markdown())

|    | taskID   | task                         | predecessorTaskIDs     |   bestCaseHours |   expectedHours |   worstCaseHours |   projectManager |   frontendDeveloper |   backendDeveloper |   dataScientist |   dataEngineer |   Rate_Hour |   Quantity_People |   Total_Costs |   PM_Costs |   Dev_Team_Costs |   Maximum Reduction |   Crashing |   Crash_Cost_Hour |
|---:|:---------|:-----------------------------|:-----------------------|----------------:|----------------:|-----------------:|-----------------:|--------------------:|-------------------:|----------------:|---------------:|------------:|------------------:|--------------:|-----------:|-----------------:|--------------------:|-----------:|------------------:|
|  0 | A        | Describe product             |                        |           20    |              20 |             20   |                1 |                   0 |                  0 |               0 |              0 |          60 |                 1 |          1200 

In [286]:
# import file 002. Input Tables\design_effort_details.xlsx
input_file_path = '002. Input Tables/design_effort_details.xlsx'
df = read_data(input_file_path)

# print column names
print(df.columns)

# Keep columns 'Design_ID', 'projectManager', 'frontendDeveloper', 'backendDeveloper', 'dataScientist', 'dataEngineer'
# in a df_temp dataframe
df_temp = df[['Design_ID', 'projectManager', 'frontendDeveloper', 'backendDeveloper', 'dataScientist', 'dataEngineer']]

# pivot the df_temp dataframe to get the desired format
# the columns 'projectManager', 'frontendDeveloper', 'backendDeveloper', 'dataScientist', 'dataEngineer'
# should be combined in one called 'Role' and the values should be in a column called 'Effort'
df_temp = df_temp.melt(id_vars='Design_ID', var_name='Role', value_name='Effort')

# if effort is 0 then remove the row
df_temp = df_temp[df_temp['Effort'] != 0]

# aggregate the Design_ID as a list by Role
# keep the effort column
df_temp = df_temp.groupby('Role').agg({ 'Effort': 'sum', 'Design_ID': list}).reset_index()

# remove the list brackets from the Design_ID column
df_temp['Design_ID'] = df_temp['Design_ID'].apply(lambda x: ', '.join([str(i) for i in x]))

# sort by Effort in descending order
df_temp = df_temp.sort_values('Effort', ascending=False)

# display data as markdown table
print(df_temp.to_markdown(index=False))

# export markdown table to '003. Output Files/table_02_design_effort_summary_role.md'
# using pandas to_markdown function
output_file_path = '003. Output Files/table_02_design_effort_summary_role.md'
with open(output_file_path, 'w') as f:
    f.write(df_temp.to_markdown(index=False))

print('\n')


# now using dataframe df keep the columns
# 'Design_ID', 'Design_Description', 'Details'
# display data as markdown table
# do not show the index
df_temp = df[['Design_ID', 'Design_Description', 'Details']]
print(df_temp.to_markdown(index=False))

# export markdown table to '003. Output Files/table_05_design_effort_details.md'
output_file_path = '003. Output Files/table_05_design_effort_details.md'

# export the dataframe to markdown file
with open(output_file_path, 'w') as f:
    f.write(df_temp.to_markdown(index=False))


Index(['Design_ID', 'Design_Description', 'Details', 'projectManager',
       'frontendDeveloper', 'backendDeveloper', 'dataScientist',
       'dataEngineer'],
      dtype='object')
| Role              |   Effort | Design_ID                                                       |
|:------------------|---------:|:----------------------------------------------------------------|
| backendDeveloper  |      180 | F01, F02, F04, F05, F06, F07, F08, F10, F11, F12, F13, T05, T06 |
| dataEngineer      |      160 | F11, F12, T01, T02, T03, T04, T05, T06                          |
| frontendDeveloper |      150 | F01, F02, F04, F05, F06, F07, F08, F10, F13                     |
| dataScientist     |       80 | F11, F12                                                        |


| Design_ID   | Design_Description               | Details                                                                                                                                                         |
|:-------

In [287]:
# load the table 'project_plan.xlsx' from '002. Input Tables'
input_file_path = '002. Input Tables/project_plan.xlsx'
df = read_data(input_file_path)

# more trailing characters from all columns in the dataframe
# apply lambda if x is string, then remove trailing characters
def remove_trailing_characters(df):
    df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    return df

# remove trailing characters from all columns in the dataframe
df = remove_trailing_characters(df)

# remove ',' from predecessorTaskIDs and add element to a list
# removving trailing spaces
# using apply() + lambda
df['predecessorTaskIDs'] = df['predecessorTaskIDs'].apply(lambda x: x.split(',')).tolist()

# remove trailing spaces
df['predecessorTaskIDs'] = df['predecessorTaskIDs'].apply(lambda x: [i.strip() for i in x]).tolist()

In [297]:
# create dictionary with data
# taskID and task
taskID_task = dict(zip(df['taskID'], df['task']))
activities_list = taskID_task.keys()

# create a dictionary with data
# taskID and expected_duration
taskID_expectedHours = dict(zip(df['taskID'], df['expectedHours']))
activities = taskID_expectedHours

# create a list to the element of the column predecessorTaskIDs
predecessorTaskIDs = df['predecessorTaskIDs'].tolist()

# create a dictionary with data
# taskID and predecessorTaskIDs
taskID_predecessorTaskIDs = dict(zip(df['taskID'], df['predecessorTaskIDs']))
precedences = taskID_predecessorTaskIDs

# print activities_list,activities,precedences
print(activities_list)
print(activities)
print(precedences)

# for all the values in the precendences dictionary
# that are '' replace by empty list
for key, value in precedences.items():
    if value == ['']:
        precedences[key] = []

# print precedences
print(precedences)

dict_keys(['A', 'B', 'C', 'D', 'D1', 'D2', 'D3', 'D4.0', 'D4.1', 'D4.2', 'D4.3', 'D4.4', 'D4.5', 'D5', 'D6', 'D7', 'D8', 'E', 'F', 'G', 'H'])
{'A': 20, 'B': 40, 'C': 20, 'D': 0, 'D1': 40, 'D2': 40, 'D3': 40, 'D4.0': 0, 'D4.1': 150, 'D4.2': 180, 'D4.3': 80, 'D4.4': 160, 'D4.5': 0, 'D5': 20, 'D6': 40, 'D7': 40, 'D8': 20, 'E': 30, 'F': 15, 'G': 30, 'H': 15}
{'A': [''], 'B': [''], 'C': ['A'], 'D': ['A'], 'D1': ['D'], 'D2': ['D1'], 'D3': ['D1'], 'D4.0': ['D2', 'D3'], 'D4.1': ['D4.0'], 'D4.2': ['D4.0'], 'D4.3': ['D4.0'], 'D4.4': ['D4.0'], 'D4.5': ['D4.1', 'D4.2', 'D4.3', 'D4.4'], 'D5': ['D4.5'], 'D6': ['D5'], 'D7': ['D6'], 'D8': ['D5', 'D7'], 'E': ['B', 'C'], 'F': ['D8', 'E'], 'G': ['A', 'D8'], 'H': ['F', 'G']}
{'A': [], 'B': [], 'C': ['A'], 'D': ['A'], 'D1': ['D'], 'D2': ['D1'], 'D3': ['D1'], 'D4.0': ['D2', 'D3'], 'D4.1': ['D4.0'], 'D4.2': ['D4.0'], 'D4.3': ['D4.0'], 'D4.4': ['D4.0'], 'D4.5': ['D4.1', 'D4.2', 'D4.3', 'D4.4'], 'D5': ['D4.5'], 'D6': ['D5'], 'D7': ['D6'], 'D8': ['D5', 'D7'], '

In [291]:
# Critical Path Analysis based on Thomas W. Miller solution
# Implemented using activities dictionary with derived start_times and end_times
# for the expectedHours of each activity

In [292]:
# Create the LP problem
prob = LpProblem("Critical_Path", LpMinimize)

# Create the LP variables
start_times = {activity: LpVariable(f"start_{activity}", 0, None) for activity in activities_list}
end_times = {activity: LpVariable(f"end_{activity}", 0, None) for activity in activities_list}

# Create the LP variables
start_times = {activity: LpVariable(f"start_{activity}", 0, None) for activity in activities_list}
end_times = {activity: LpVariable(f"end_{activity}", 0, None) for activity in activities_list}

# Set the objective function
prob += lpSum([end_times[activity] for activity in activities_list]), "minimize_end_times"

# Add the constraints
for activity in activities_list:
    prob += end_times[activity] == start_times[activity] + activities[activity], f"{activity}_duration"
    for predecessor in precedences[activity]:
        if predecessor != []:
            prob += start_times[activity] >= end_times[predecessor], f"{activity}_predecessor_{predecessor}"

# Solve the LP problem
status = prob.solve()

# Print the results
print("Critical Path time:")
for activity in activities_list:
    if (start_times[activity].value()) == 0:
        print(f"{activity} starts at time 0")
    if (end_times[activity].value()) == max([end_times[activity].value() for activity in activities_list]):
        print(f"{activity} ends at {end_times[activity].value()} hours in duration")
        print(f"Or, {activity} ends at {end_times[activity].value()/8} work weeks in duration")

# store first the results in a list
results = []
# iterate over the activities_list
for activity in activities_list:
    # store the results in a list
    results.append([activity, start_times[activity].value(), end_times[activity].value()])

# add the results to the data frame
df_results = pd.DataFrame(results, columns=['Activity', 'Start', 'End'])

# display data as markdown table
print(df_results.to_markdown(index=False))

# export output/listing as plain text files
# problem description
output_file_path = '003. Output Files/problem_description.txt'


Critical Path time:
A starts at time 0
B starts at time 0
H ends at 445.0 hours in duration
Or, H ends at 55.625 work weeks in duration
| Activity   |   Start |   End |
|:-----------|--------:|------:|
| A          |       0 |    20 |
| B          |       0 |    40 |
| C          |      20 |    40 |
| D          |      20 |    20 |
| D1         |      20 |    60 |
| D2         |      60 |   100 |
| D3         |      60 |   100 |
| D4.0       |     100 |   100 |
| D4.1       |     100 |   250 |
| D4.2       |     100 |   280 |
| D4.3       |     100 |   180 |
| D4.4       |     100 |   260 |
| D4.5       |     280 |   280 |
| D5         |     280 |   300 |
| D6         |     300 |   340 |
| D7         |     340 |   380 |
| D8         |     380 |   400 |
| E          |      40 |    70 |
| F          |     400 |   415 |
| G          |     400 |   430 |
| H          |     430 |   445 |


In [293]:
# print problem description
with open(output_file_path, 'w') as f:
    f.write("Critical Path Analysis based on Thomas W. Miller solution\n")
    f.write("Implemented using activities dictionary with derived start_times and end_times\n")
    f.write("for the expectedHours of each activity\n")
    f.write("\n")
    f.write("Create the LP problem\n")
    f.write("Create the LP variables\n")
    f.write("Set the objective function\n")
    f.write("Add the constraints\n")
    f.write("Solve the LP problem\n")
    f.write("Print the results\n")
    f.write("\n")
    f.write("Critical Path time:\n")
    for activity in activities_list:
        if (start_times[activity].value()) == 0:
            f.write(f"{activity} starts at time 0\n")
        if (end_times[activity].value()) == max([end_times[activity].value() for activity in activities_list]):
            f.write(f"{activity} ends at {end_times[activity].value()} hours in duration\n")
            f.write(f"Or, {activity} ends at {end_times[activity].value()/8} work days duration\n")
    f.write("\n")
    f.write("Problem full formulation\n")
    f.write(str(prob))

In [294]:
# add a new column to the data frame
# Ref_Date = '04.20.2023' in the format 'mm.dd.yyyy'
# as date
df_results['Ref_Date'] = '04.22.2023'

# convert the start and end from hours to work days considering 8 hours per day
df_results['Start'] = df_results['Start'] / 8
df_results['End'] = df_results['End'] / 8

# calculate the duration in work days between the start and end
df_results['Duration'] = df_results['End'] - df_results['Start']

# calculate the start_date and end_date considering the Ref_Date
# end_date should contain also the weekends
df_results['Start_Date'] = pd.to_datetime(df_results['Ref_Date']) \
    + pd.to_timedelta(df_results['Start'], unit='D')

df_results['End_Date'] = pd.to_datetime(df_results['Ref_Date']) \
    + pd.to_timedelta(df_results['Start'], unit='D')\
    + pd.to_timedelta(df_results['Duration'], unit='D')

# display data as markdown table
print(df_results.to_markdown(index=False))


| Activity   |   Start |    End | Ref_Date   |   Duration | Start_Date          | End_Date            |
|:-----------|--------:|-------:|:-----------|-----------:|:--------------------|:--------------------|
| A          |    0    |  2.5   | 04.22.2023 |      2.5   | 2023-04-22 00:00:00 | 2023-04-24 12:00:00 |
| B          |    0    |  5     | 04.22.2023 |      5     | 2023-04-22 00:00:00 | 2023-04-27 00:00:00 |
| C          |    2.5  |  5     | 04.22.2023 |      2.5   | 2023-04-24 12:00:00 | 2023-04-27 00:00:00 |
| D          |    2.5  |  2.5   | 04.22.2023 |      0     | 2023-04-24 12:00:00 | 2023-04-24 12:00:00 |
| D1         |    2.5  |  7.5   | 04.22.2023 |      5     | 2023-04-24 12:00:00 | 2023-04-29 12:00:00 |
| D2         |    7.5  | 12.5   | 04.22.2023 |      5     | 2023-04-29 12:00:00 | 2023-05-04 12:00:00 |
| D3         |    7.5  | 12.5   | 04.22.2023 |      5     | 2023-04-29 12:00:00 | 2023-05-04 12:00:00 |
| D4.0       |   12.5  | 12.5   | 04.22.2023 |      0     | 2023

In [295]:
# lets create a tag for the roles resposible for each task
# using the df dataframe
# keep the columns 'taskID', 'task', 'projectManager', 'frontendDeveloper', 'backendDeveloper', 'dataScientist', 'dataEngineer'
df_temp = df[['taskID', 'task', 'projectManager', 'frontendDeveloper', 'backendDeveloper', 'dataScientist', 'dataEngineer']]

# pivot the df_temp dataframe to get the desired format
# the columns 'projectManager', 'frontendDeveloper', 'backendDeveloper', 'dataScientist', 'dataEngineer'
# should be combined in one called 'Role' and the values should be in a column called 'Effort'
df_temp = df_temp.melt(id_vars=['taskID', 'task'], var_name='Role', value_name='Name')

# remove the rows where the Name is 0
df_temp = df_temp[df_temp['Name'] != 0]

# aggreagte taskID and task as a list by Role
# do not keep Name
df_temp = df_temp.groupby(['taskID', 'task']).agg({'Role': list}).reset_index()

# each member of the Role list should be simplified using the following dictionary
# projectManager: PM
# frontendDeveloper: FE_Dev
# backendDeveloper: BE_Dev
# dataScientist: DS
# dataEngineer: DE

# create a dictionary with the mapping
mapping = {
    'projectManager': 'PM',
    'frontendDeveloper': 'FE_Dev',
    'backendDeveloper': 'BE_Dev',
    'dataScientist': 'DS',
    'dataEngineer': 'DE'
}

# apply the mapping to the Role column
df_temp['Role'] = df_temp['Role'].apply(lambda x: [mapping[i] for i in x])

# remove the list brackets from the Role column
df_temp['Role'] = df_temp['Role'].apply(lambda x: ', '.join([str(i) for i in x]))

# display data as markdown table
print(df_temp.to_markdown(index=False))

# call df_temp as tags
tags = df_temp

| taskID   | task                         | Role                       |
|:---------|:-----------------------------|:---------------------------|
| A        | Describe product             | PM                         |
| B        | Develop marketing strategy   | PM                         |
| C        | Design brochure              | PM                         |
| D1       | Requirements analysis        | PM, FE_Dev, BE_Dev, DS, DE |
| D2       | Software design              | PM, FE_Dev, BE_Dev, DS, DE |
| D3       | System design                | PM, BE_Dev, DE             |
| D4.1     | Coding - Front-end dev       | FE_Dev                     |
| D4.2     | Coding - Back-end dev        | BE_Dev                     |
| D4.3     | Coding - Data Scientist      | DS                         |
| D4.4     | Coding - Data Engineer       | DE                         |
| D5       | Write documentation          | PM, FE_Dev, BE_Dev, DS, DE |
| D6       | Unit testing                 | FE_Dev,

In [296]:
# join the df_results and tags dataframes
# remove duplicate columns
df_results = df_results.merge(tags, left_on='Activity', right_on='taskID', how='left')

# create a Gannt chart using plotly
import plotly.express as px

# create a figure , showing arrows between the activities
fig = px.timeline(df_results, x_start='Start_Date'
                  , x_end='End_Date'
                  , y='Activity'
                  , title='Gannt Chart - expected timeline - Project Plan \
                    <br><sup>Total duration: '\
                    +f'{total_duration} days'+'</sup>'
                  , color='Role'
                  , labels={'Activity': 'Task'}
                  , width=1000
                  , height=600
                  , template='plotly_white'
                  , category_orders={'Activity': activities_list}
                  , hover_name='Activity'
                  , hover_data={'task': True, 'Start': True, 'End': True, 'Duration': True, }
                  )

# show the figure
fig.show()

# export the figure to '003. Output Files/fig_01_gannt_chart'
output_file_path = '003. Output Files/fig_01_gannt_chart'

# export the figure to html and image
fig.write_html(output_file_path + '.html')
#fig.write_image(output_file_path + '.png')
